In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utils import epsilon_greedy_policy, initialize_q_table, rec_state, rec_state_path, get_best_params
from RLOpt import ObjModel, train

import model_defect_recognition_data_Loader
import tensorflow as tf
import shell_model_defect_recognition

In [5]:
# Defect recognition model part

# model parameters space
params = {
            'activation_input': ['relu', 'sigmoid', 'tanh'],
            'activation_output': ['sigmoid', 'relu'],
            'optimizer': [tf.keras.optimizers.Adam(), tf.keras.optimizers.RMSprop()]
}

In [3]:
# Data init
txt_path = r'C:\Users\Ruslan\Documents\TVEMA\DataSet\dataset_copy\Group5\53'
test_way = model_defect_recognition_data_Loader.TxtDataSetFilesLoading(txt_path, '.txt', np.uint8,
                                            r"C:\Users\Ruslan\Documents\TVEMA\DataSet\dataset_copy\Group5\data_info.json",
                                            True)

images, labels = test_way.load_files("defect", "all", "multiple_ch")

# Scaling
images = images / 255

# decrease dataset size because of memory limit
images = images[:10000]
labels = labels[:10000]

# Splitting
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2)

data = (train_images, train_labels, test_images, test_labels)

print("Train images shape: {}".format(data[0].shape))
print("Test images shape: {}".format(data[2].shape))

print("Train labels shape: {}".format(data[1].shape))
print("Test labels shape: {}".format(data[3].shape))

Found .txt files (60449) in C:\Users\Ruslan\Documents\TVEMA\DataSet\dataset_copy\Group5\53
Found .json files (1) in C:\Users\Ruslan\Documents\TVEMA\DataSet\dataset_copy\Group5\53
Created defect catalog info at: C:\Users\Ruslan\Documents\TVEMA\DataSet\dataset_copy\Group5\data_info.json
-------------------------------- loading start ---------------------------------


100%|██████████| 60449/60449 [01:53<00:00, 534.79it/s]  


Total files count in dir (.txt): 60449
Loaded images files with extension (.txt) count: 12923
Labels count: 12923
Skipped files (.txt): 47526
Train images shape: (8000, 4, 150, 128)
Test images shape: (2000, 4, 150, 128)
Train labels shape: (8000, 4)
Test labels shape: (2000, 4)


In [6]:
env = ObjModel(shell_model_defect_recognition.ModelDefectRecognition, params, data)     # environment
# Training parameters
learning_rate = 1          # Learning rate
max_steps = int(env.observation_space_n / 2)            # Max steps per episode
gamma = 0.9                 # Discounting rate

# Exploration parameters
epsilon = 1.0
decay_rate = 0.005            # Exponential decay rate for exploration prob

In [ ]:
state_space = env.observation_space_n # TODO
action_space = env.action_space_n # TODO

print(env.observation_space_n)

Qtable_init_s = initialize_q_table(state_space, action_space)

Qtable_s, episod_rewards_s = train(epsilon, decay_rate, env, max_steps, Qtable_init_s, gamma, learning_rate)

params = get_best_params(Qtable_s, env, calc_acc=True)

In [ ]:
accs = [i[1] for i in params]
print(accs, 'best acc:', np.max(accs))

In [ ]:
print('best params:', params[np.argmax(accs)][0])

In [ ]:
plt.plot(np.cumsum(episod_rewards_s[1]))
plt.plot(episod_rewards_s[0])
plt.plot(episod_rewards_s[1])